In [1]:
# Importing necessary files.
import numpy as np
import pandas as pd
import seaborn as sns
import glob, os

# Added dask to accelerate data handling.
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Collect all dates in a tuple.
dates_tuple = tuple([f'202402'+str(i) if i>=10 else '2024020'+str(i)  for i in range(1,30)] + 
                    [f'202403'+str(i) if i>=10 else '2024030'+str(i)  for i in range(1,32)])
# Define subfolders shorthand.
subfolders_dic = {'match':'raw_match_parquet',
                  'odds':'raw_odds_parquet',
                  'pbp':'raw_point_by_point_parquet',
                  'stats':'raw_statistics_parquet',
                  'power':'raw_tennis_power_parquet',
                  'votes':'raw_votes_parquet'
                  }
# Define function to render addresses.
def render_addresses(subfolder:str, date_arr = None, kind:str = ""):
    """
    Function to render addresses.
    It takes a subfolder, array of dates and kind of file as inputs.
    """
    result = []
    date_arr = dates_tuple if not date_arr else date_arr
    if kind:
        result = [file for dt in date_arr
        for file in glob.glob(f"Tennis Schema\\{dt}\\data\\raw\\{subfolders_dic[subfolder]}\\{kind}_[!s]*.parquet")]
    else:
        result = [file for dt in date_arr
        for file in glob.glob(f"Tennis Schema\\{dt}\\data\\raw\\{subfolders_dic[subfolder]}\\*.parquet")]
    return result

In [ ]:
# How many players are there in the dataset?
players_id_df = dd.read_parquet(render_addresses('match', kind='*_team'),columns=['player_id'])

In [207]:
players_id_df.compute()

,player_id
0,192013
0,273680
0,77223
0,88992
0,248846
...,...
0,375214
0,215205
0,398073
0,303293


In [217]:
# The average height of players.
players_height_df =dd.read_parquet(render_addresses('match', kind='*_team'),columns=['player_id','height'])

In [219]:
players_height_df.drop_duplicates(subset='player_id', keep='first').compute()

,player_id,height
0,197546,1.85
0,340741,1.88
0,17046,1.80
0,130690,NaN
0,156826,1.85
...,...,...
0,46651,1.73
0,265808,1.75
0,383458,NaN
0,383428,NaN


In [54]:
# Extracting the necessary coulumns for the player with highest number of wins.
event_addresses = render_addresses('match', date_arr = dates_tuple,kind='event')
home_addresses = render_addresses('match', date_arr = dates_tuple, kind='home_team')
away_addresses = render_addresses('match', date_arr = dates_tuple, kind='away_team')

match_df = dd.read_parquet(event_addresses, columns=['match_id','winner_code'])
home_df = dd.read_parquet(home_addresses, columns=['match_id','player_id','full_name'])
away_df = dd.read_parquet(away_addresses, columns=['match_id','player_id','full_name'])

In [61]:
# Extracting the results into csv for faster manipulation with pandas.
with ProgressBar():
    result = away_df.compute()

[########################################] | 100% Completed | 78.32 s


In [85]:
# Opening folders and then dropping duplicates.
match_df = pd.read_csv("CSVs\\3_match_df.csv")
home_df = pd.read_csv("CSVs\\3_home_df.csv")
away_df = pd.read_csv("CSVs\\3_away_df.csv")
match_df.drop_duplicates(subset='match_id', inplace=True)
home_df.drop_duplicates(subset='match_id', inplace=True)
away_df.drop_duplicates(subset='match_id', inplace=True)

In [ ]:
# Data Wranglign with grouping and filtering.
winner_home_df = match_df.query('winner_code == 1').merge(home_df, on='match_id', how='inner')
winner_home_df = winner_home_df.drop(columns=['winner_code'])
winner_away_df = match_df.query('winner_code == 2').merge(away_df, on='match_id', how='inner')
winner_away_df = winner_away_df.drop(columns=['winner_code'])
winners_df = pd.concat([winner_home_df,winner_away_df])
winners_df = winners_df.dropna().drop_duplicates()
winners_df.player_id = winners_df.player_id.astype(int)

In [97]:
winners_df.groupby('player_id').agg({'match_id':'count',
                                    'full_name':'min'}).rename(
                                    columns={'match_id':'matches_won'}).sort_values(
                                    'matches_won',ascending=False)

,matches_won,full_name
player_id,,
50901,29,"Popko, Dmitry"
231620,22,"Chidekh, Clement"
202572,21,"Gengel, Marek"
230049,21,"Jianu, Filip Cristian"
82133,20,"Dellien Velasco, Murkel Alejandro"
...,...,...
175258,1,"Benaissa, Amira"
175160,1,"Lavino, Irene"
59213,1,"Perez Garcia, Maria Paulina"


In [108]:
# Extracting the time results into csv for faster manipulation with pandas.
time_df = dd.read_parquet(render_addresses('match', kind='time'))

with ProgressBar():
    result = time_df.compute()

[########################################] | 100% Completed | 145.17 s


In [158]:
time_df = pd.read_csv("CSVs\\4_time_df.csv")
time_df.drop_duplicates(subset='match_id', inplace=True)
time_df.dropna(subset=['period_1'], inplace=True)
time_df.drop(columns=['period_4','period_5','current_period_start_timestamp'], inplace=True)
time_df['duration'] = time_df[['period_1', 'period_2', 'period_3']].sum(axis=1, skipna=True).div(60).round(2)
time_df.sort_values(by=['duration'],ascending=False)

,match_id,period_1,period_2,period_3,duration
5799,12063611,167352.0,169438.0,NaN,5613.17
5795,12063587,159144.0,161086.0,NaN,5337.17
30980,12185562,4163.0,84588.0,88380.0,2952.18
17230,12121829,172605.0,810.0,NaN,2890.25
3803,12054403,2170.0,80162.0,81551.0,2731.38
...,...,...,...,...,...
1026,12039701,77.0,26.0,NaN,1.72
35668,12213484,3.0,5.0,NaN,0.13
6344,12064959,2.0,6.0,NaN,0.13
5853,12063889,4.0,4.0,NaN,0.13


In [146]:
with ProgressBar():
    res = dd.read_parquet(render_addresses('match', kind='event')).compute()

[########################################] | 100% Completed | 170.04 s


In [163]:
# Data entry might be wrong!
# Look at https://www.sofascore.com/fr/tennis/match/o-gavrila-carolina-alves/OLvsnbR
res[res.match_id == 12177252]

,match_id,first_to_serve,home_team_seed,away_team_seed,custom_id,winner_code,default_period_count,start_datetime,match_slug,final_result_only
0,12177252,1,<NA>,<NA>,kLysfnxc,1,3,1711017000,beckley-caruso,False
0,12177252,1,<NA>,<NA>,kLysfnxc,1,3,1711017000,beckley-caruso,False
0,12177252,1,<NA>,<NA>,kLysfnxc,1,3,1711017000,beckley-caruso,False


In [10]:
with ProgressBar():
    res = dd.read_parquet(render_addresses('match', kind="event"), columns = ['default_period_count']).compute()

[########################################] | 100% Completed | 89.88 s


In [12]:
res.value_counts()

default_period_count
3                       35053
Name: count, dtype: int64

In [ ]:
# Opening folders and then dropping duplicates.
match_df = pd.read_csv("CSVs\\3_match_df.csv")
home_df = pd.read_csv("CSVs\\3_home_df.csv")
away_df = pd.read_csv("CSVs\\3_away_df.csv")
match_df.drop_duplicates(subset='match_id', inplace=True)
home_df.drop_duplicates(subset='match_id', inplace=True)
away_df.drop_duplicates(subset='match_id', inplace=True)

[########################################] | 100% Completed | 731.59 ms


In [48]:
# Extracting the necessary coulumns for the player with highest number of wins.
home_addresses = render_addresses('match', date_arr = dates_tuple, kind='home_team')
away_addresses = render_addresses('match', date_arr = dates_tuple, kind='away_team')

home_df = dd.read_parquet(home_addresses, columns=['player_id','full_name', 'current_rank', 'total_prize','country'])
away_df = dd.read_parquet(away_addresses, columns=['player_id','full_name', 'current_rank', 'total_prize','country'])

In [52]:
with ProgressBar():
    res = away_df.compute()

[########################################] | 100% Completed | 71.24 s


In [53]:
res

,player_id,full_name,current_rank,total_prize,country
0,192013,"Auger-Aliassime, Felix",30,10166964,Canada
0,273680,Flavio Cobolli,69,808536,Italy
0,77223,"Martinez, Pedro",101,2732689,Spain
0,88992,"Muller, Alexandre",81,1178035,France
0,248846,"Mayot, Harold",132,424547,France
...,...,...,...,...,...
0,72324,"Doria, Luciano",1442,18936,Argentina
0,217437,"Soriano Barrera, Adria",465,41592,Colombia
0,108559,"Taberner, Carlos",358,902241,Spain
0,74679,"Simonsson, Fred",None,52868,Sweden


In [54]:
# Opening folders and then dropping duplicates.
home_df = pd.read_csv("CSVs\\6_home_df.csv")
away_df = pd.read_csv("CSVs\\6_away_df.csv")
players_df = pd.concat([home_df, away_df])
players_df.drop_duplicates(subset='player_id', inplace=True)

In [78]:
def compute_success_scores(players, w1=0.5, w2=0.5):
    """
    players: DataFrame or list of dicts with columns/keys:
      - 'current_rank'   (int, 1 = best)
      - 'total_prize'    (float, in USD)
    Returns a DataFrame with a new column 'success_score' in [0,1].
    """
    df = pd.DataFrame(players).copy()

    # 1) normalize rank: invert, then scale so best rank → 1.0, worst → 0.0
    #    (we assume ranks start at 1)
    inv_rank = 1.0 / df['current_rank']
    df['norm_rank'] = (inv_rank - inv_rank.min()) / (inv_rank.max() - inv_rank.min())

    # 2) normalize prize: log(1 + prize) to dampen skew, then min–max scale
    log_prize = np.log1p(df['total_prize'])
    df['norm_prize'] = (log_prize - log_prize.min()) / (log_prize.max() - log_prize.min())

    # 3) weighted sum
    df['success_score'] = w1 * df['norm_rank'] + w2 * df['norm_prize']

    return df
compute_success_scores(players_df,w1= 0.5, w2=0.5).sort_values(by='success_score',ascending=False).dropna().iloc[:100].groupby('country').sum()['success_score'].sort_values(ascending=False)

# compute_success_scores(players_df,w1= 0.5, w2=0.5).sort_values(by='success_score',ascending=False).dropna().iloc[:100].query("country == 'France'")

country
USA               4.797982
France            4.032480
Russia            3.950585
Spain             2.360521
Italy             2.132062
Serbia            1.767280
Poland            1.764142
Czech Republic    1.718508
Croatia           1.620453
Germany           1.361629
Kazakhstan        1.289573
United Kingdom    1.268509
Romania           1.228186
Canada            1.222683
Belarus           1.124491
Greece            0.909786
Denmark           0.888685
Japan             0.841659
Ukraine           0.832436
Latvia            0.831243
Belgium           0.827923
China             0.809801
Argentina         0.780775
Hungary           0.760291
Tunisia           0.478613
Bulgaria          0.461452
Norway            0.452776
Switzerland       0.445516
Australia         0.438472
Austria           0.435112
Brazil            0.400496
Chinese Taipei    0.393092
Chile             0.383430
Georgia           0.381789
Portugal          0.381089
Name: success_score, dtype: float64

In [47]:
with ProgressBar():
    res = dd.read_parquet(render_addresses('stats'), columns = ['match_id','period','statistic_name','home_stat','away_stat']).compute()

[#########                               ] | 24% Completed | 24.02 ss


KeyboardInterrupt: 

In [31]:
df_.query("statistic_name == 'aces'")

,match_id,period,statistic_category_name,statistic_name,home_stat,away_stat,compare_code,statistic_type,value_type,home_value,away_value,home_total,away_total
0,11998455,ALL,service,aces,5,19,2,positive,event,5,19,NaN,NaN
20,11998455,1ST,service,aces,4,6,2,positive,event,4,6,NaN,NaN
37,11998455,2ND,service,aces,1,5,2,positive,event,1,5,NaN,NaN
53,11998455,3RD,service,aces,0,8,2,positive,event,0,8,NaN,NaN
0,11998773,ALL,service,aces,7,8,2,positive,event,7,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,12047771,3RD,service,aces,0,0,3,positive,event,0,0,NaN,NaN
0,12048717,ALL,service,aces,10,4,1,positive,event,10,4,NaN,NaN
20,12048717,1ST,service,aces,1,0,1,positive,event,1,0,NaN,NaN
37,12048717,2ND,service,aces,2,0,1,positive,event,2,0,NaN,NaN
